In [ ]:
import pandas as pd
import numpy as np
import pickle
from tqdm.notebook import tqdm

In [ ]:
df_ednet = pd.read_pickle('./data/ednet/preprocessed_total_data.pkl').dropna()
df_ednet

In [ ]:
# duplicate answer question data remove
from collections import defaultdict

null_dict = defaultdict(set)

for row in tqdm(df_ednet.itertuples()):
    question_id = row.question_id
    is_correct = row.corretness
    correct_answer = row.correct_answer

    if is_correct == 1:
        null_dict[question_id].add(correct_answer)
    elif is_correct == 0:
        null_dict[question_id].add(correct_answer)

null_list = [q for q, answers in null_dict.items() if len(answers) > 1]


In [ ]:
# baseline id_dic_construction
def id_dic_construction(x):
    corresponding_dic = {}
    for dic_index in range(len(x)):
        corresponding_dic[x[dic_index]] = dic_index
    return corresponding_dic

In [ ]:
question_info = pd.read_csv("./data/ednet/questions.csv")
question_info


In [ ]:
# baseline id_dic

# Extract question_info 
question_info = pd.read_csv("./data/ednet/questions.csv")
for i, q in enumerate(question_info['question_id']):
    question_info['question_id'][i] = q[1:]
question_info
print(question_info.columns)
question_info = question_info.drop_duplicates()

question_info = question_info.values

# Extract question_id
question_id = np.unique(question_info[:, 0])
question_dic = id_dic_construction(question_id)

# Extract skill_id
for i, kc in enumerate(question_info):
    question_info[i, 5] = list(map(int, kc[5].split(';')))
skill = []
for i in range(len(question_info)):
    skill += question_info[i, 5]
skill = np.unique(np.array(skill).astype('int64'))
skill_dic = id_dic_construction(skill)

# question, skill renaming with dictionary
for i in range(len(question_info)):
    question_info[i, 0] = question_dic[question_info[i, 0]]
    question_info[i, 5] = [skill_dic[skill] for skill in question_info[i, 5]]
'''
for i in range(len(question_info)):
    question_info[i, 0] = question_dic[question_info[i, 0]]
    for n in range(len(question_info[i, 1])):
        question_info[i, 1][n] = skill_dic[question_info[i, 1][n]]
'''
question_info

In [ ]:
with open ("./data/ednet/question_dic.pkl", "wb") as f:
    pickle.dump(question_dic, f)
with open ("./data/ednet/skill_dic.pkl", "wb") as f:
    pickle.dump(skill_dic, f)   

In [ ]:
# Make baseline q_matrix
tmp_question_skill_cell = []
for i in tqdm(range(len(question_info))):
    q_id = question_info[i, 0]
    skill = question_info[i, 5]
    for kc in np.array(skill).astype('int64'):
        tmp_question_skill_cell.append([q_id, kc])
tmp_question_skill_cell = np.array(tmp_question_skill_cell)
exercise_num = len(question_dic)
skill_num = len(skill_dic)
tmp_exercise_id = tmp_question_skill_cell[:, 0]
tmp_skill_id = tmp_question_skill_cell[:, 1]
baseline_q_matrix = np.zeros([exercise_num, skill_num])
for i in tqdm(range(len(tmp_question_skill_cell))):
    baseline_q_matrix[int(tmp_exercise_id[i]), int(tmp_skill_id[i])] = 1
baseline_q_matrix

In [ ]:
with open ("./data/ednet/q_matrix_ednet.pkl", "wb") as f:
    pickle.dump(baseline_q_matrix, f)

In [ ]:
question_dic = pd.read_pickle("./data/ednet/question_dic.pkl")
skill_dic = pd.read_pickle("./data/ednet/skill_dic.pkl")

In [ ]:
# Problem solving rate of each student

from collections import defaultdict

problem_solving_rate = defaultdict(int)

grouped = df_ednet.groupby(df_ednet.iloc[:, 0])

for student, group in tqdm(grouped):
    unique_questions = group.iloc[:, 1].unique()
    problem_solving_rate[student] = len(unique_questions) / len(question_dic)

dict(problem_solving_rate)


In [ ]:
with open("./data/ednet/problem_solving_rate.pkl", "wb") as f:
    pickle.dump(problem_solving_rate, f)

In [ ]:
problem_solving_rate = pd.read_pickle("./data/ednet/problem_solving_rate.pkl")

In [ ]:
# Percentage of total correct answers for each student

from collections import defaultdict
correct_rate = defaultdict(int)


grouped = df_ednet.groupby(df_ednet.iloc[:, 0])

for student, group in tqdm(grouped):
    if len(group.iloc[:, 6]) == 0:
        correct_rate[student] = 0
    else:
        correct_rate[student] = sum(group.iloc[:, 6])/len(group.iloc[:, 6])

correct_rate

In [ ]:
with open("./data/ednet/correct_rate.pkl", "wb") as f:
    pickle.dump(correct_rate, f)

In [ ]:
correct_rate = pd.read_pickle("./data/ednet/correct_rate.pkl")

In [ ]:
# A dictionary with a list of students who solved each problem

from collections import defaultdict
problem_student = defaultdict(int)

grouped = df_ednet.groupby(df_ednet.iloc[:, 1])


for question, group in tqdm(grouped):
    unique_students = group.iloc[:, 0].unique()
    problem_student[question] = unique_students
problem_student

In [ ]:
with open("./data/ednet/problem_student.pkl", "wb") as f:
    pickle.dump(problem_student, f)

In [ ]:
problem_student = pd.read_pickle("./data/ednet/problem_student.pkl")

In [ ]:
# Pick leading group
leading_group = {}
problem_solving_rate_upper_bound = 0.4 # problem solving upper bound
correct_rate_upper_bound = 0.4  # correct rate upper bound

for key in tqdm(problem_student):
    tmp = np.empty((0, 3), float)
    for student in problem_student[key]:
        tmp = np.append(tmp, np.array([[student, problem_solving_rate[student], correct_rate[student]]]), axis=0)
    pro_solv_rate_sorted_idx = np.argsort(tmp[:, 1])[::-1]
    tmp = tmp[pro_solv_rate_sorted_idx]
    pro_solv_rate_up_bound = int(np.round(len(tmp[:, 1])*problem_solving_rate_upper_bound))
    tmp = tmp[:pro_solv_rate_up_bound+1]
    cor_rate_sorted_idx = np.argsort(tmp[:, 2])[::-1]
    tmp = tmp[cor_rate_sorted_idx]
    cor_rate_up_bound = int(np.round(len(tmp[:, 2])*correct_rate_upper_bound))
    tmp_leading_group = tmp[:cor_rate_up_bound+1]
    leading_group[key] = tmp_leading_group[:, 0].astype(int)

# leading_group

In [ ]:
with open ("./data/ednet/leading_group.pkl", "wb") as f:
    pickle.dump(leading_group, f)

In [ ]:
leading_group = pd.read_pickle("./data/ednet/leading_group.pkl")

In [ ]:
# Option weight extraction based on the option selection rate of the leading group
opt_dic = {}
for key, group in tqdm(df_ednet.groupby('question_id')):
    tmp_opt_list = []
    tmp_opt_dic = {}
    for i in leading_group[key]:
        filtered_df = group[group['user_id'] == i]
        tmp_opt_list += list(filtered_df['user_answer'])
        correct_answer = list(filtered_df['correct_answer'])[0]
    tmp_opt_dic["a"] = tmp_opt_list.count("a") / len(tmp_opt_list) if len(tmp_opt_list) != 0 else 0
    tmp_opt_dic["b"] = tmp_opt_list.count("b") / len(tmp_opt_list) if len(tmp_opt_list) != 0 else 0
    tmp_opt_dic["c"] = tmp_opt_list.count("c") / len(tmp_opt_list) if len(tmp_opt_list) != 0 else 0
    tmp_opt_dic["d"] = tmp_opt_list.count("d") / len(tmp_opt_list) if len(tmp_opt_list) != 0 else 0
    tmp_opt_dic[correct_answer] = 2.0
    sorted_dic = dict(sorted(tmp_opt_dic.items(), key=lambda x: x[1]))
    sorted_keys = list(sorted_dic.keys())
    tmp_opt_dic[sorted_keys[0]] = 1
    tmp_opt_dic[sorted_keys[1]] = 2
    tmp_opt_dic[sorted_keys[2]] = 3
    tmp_opt_dic[sorted_keys[3]] = 4
    opt_dic[key] = tmp_opt_dic


In [ ]:
with open ("./data/ednet/opt_dic.pkl", "wb") as f:
    pickle.dump(opt_dic, f)

In [ ]:
opt_dic = pd.read_pickle("./data/ednet/opt_dic.pkl")
# opt_dic

In [ ]:
df_ednet_values = df_ednet.values

In [ ]:
# Extract Option Weight
option_list = [opt_dic[df_ednet_values[i, 1]][df_ednet_values[i, 2]] for i in tqdm(range(len(df_ednet_values)))]
df_ednet['OptionWeight'] = option_list
df_ednet

In [ ]:
with open(f'./data/ednet/method_2/ednet.pkl', 'wb') as f:
    pickle.dump(df_ednet, f)

In [ ]:
df_ednet = pd.read_pickle("./data/ednet/method_2/ednet.pkl")
df_ednet = df_ednet.dropna()

In [ ]:
# extract if it is wrong, but the option weight is 4 ==> nothing should come out, it is normal
df_ednet[(df_ednet['corretness']==0)&(df_ednet['OptionWeight']==4)]


In [ ]:
# Save pickle file
with open(f'../data/ednet/method_2/ednet.pkl', 'wb') as f:
    pickle.dump(df_ednet, f)